# Overview
- RegNet040 5Fold ensemble
- Pseudo labeling
- 이전 대회에서 활용한 Framework을 기반으로 진행했습니다.

## Total Process
- Train Dastset이 많지 않기 때문에 Test Dataset에 대해 Pseudo labeling을 진행 후, Train Dataset에 추가하여 모델을 Training시켰습니다.
- 총 6번의 Pseudo labeling을 진행 하였습니다.
- 첫 Pseudo labeling은 최적화 과정을 거친 regnet 5fold ensemble(Public Score : 0.989)을 이용했습니다.
- 이후 동일한 Model config에서 총 5번의 Pseudo label set update를 진행하였습니다.
- 최종적으로 총 5번의 update가 된 Pseudo label set + Train Dataset에 대해 Training한 regnet 5fold ensemble을 제출 하였고 Private Score기준 1등을 달성했습니다.


## Pseudo label set sampling
- Pseudo label set의 신뢰성을 위해, 5fold ensembel logic에 대한 softmax결과값이 0.9보다 큰 sample로 한정했습니다.
- Pseudo label set에서 비율이 많은 0번 class는 random sampling(n=500)했습니다.

## Pseudo label set업데이트에 따른 Regnet 5fold ensemble Public Score
- 0step(Only Trainset) : 0.989
- 1step : 0.996
- 4step : 0.998
- 6step(Submission) : 0.999 / Private : 0.99885

# Requirements

Ubuntu 18.04, Cuda 11

- opencv-python
- numpy
- pandas
- timm
- torch==1.8.0 torchvision 0.9.0 with cuda 11.1
- natsort
- scikit-learn
- pillow
- torch_optimizer
- tqdm
- ptflops
- easydict
- matplotlib

# Directory Structure

- data
    - train.csv
    - test.csv
    - sample_submission.csv
    - train_imgs
        - 10000.jpg
        - ...
        - 10249.jpg
    - test_imgs
        - 20000.jpg
        - ...
        - 24749.jpg
- notebook
    - 1st_place_solution.ipynb
    - results
  

# Library

In [1]:
import os
import cv2
import time
import random
import logging
import easydict
import numpy as np
import pandas as pd
from tqdm import tqdm
from os.path import join as opj
from ptflops import get_model_complexity_info
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import timm
import torch
import torch.nn as nn
import torch_optimizer as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, grad_scaler
from torchvision import transforms

import warnings
warnings.filterwarnings('ignore')

# Config

In [2]:
args = easydict.EasyDict(
    {'exp_num':'0',
     
     # Path settings
     'data_path':'../data',
     'Kfold':5,
     'model_path':'results/',
     'image_type':'train_1024', 

     # Model parameter settings
     'encoder_name':'regnety_040',
     'drop_path_rate':0.2,
     
     # Training parameter settings
     ## Base Parameter
     'img_size':288,
     'batch_size':16,
     'epochs':70,
     'optimizer':'Lamb',
     'initial_lr':5e-6,
     'weight_decay':1e-3,

     ## Augmentation
     'aug_ver':2,

     ## Scheduler (OnecycleLR)
     'scheduler':'cycle',
     'warm_epoch':5,
     'max_lr':1e-3,

     ### Cosine Annealing
     'min_lr':5e-6,
     'tmax':145,

     ## etc.
     'patience':15,
     'clipping':None,

     # Hardware settings
     'amp':True,
     'multi_gpu':False,
     'logging':False,
     'num_workers':4,
     'seed':42
    })

# Utils for training and Logging

In [3]:
# Warmup Learning rate scheduler
from torch.optim.lr_scheduler import _LRScheduler
class WarmUpLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimzier(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        
        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]

# Logging
def get_root_logger(logger_name='basicsr',
                    log_level=logging.INFO,
                    log_file=None):

    logger = logging.getLogger(logger_name)
    # if the logger has been initialized, just return it
    if logger.hasHandlers():
        return logger

    format_str = '%(asctime)s %(levelname)s: %(message)s'
    logging.basicConfig(format=format_str, level=log_level)

    if log_file is not None:
        file_handler = logging.FileHandler(log_file, 'w')
        file_handler.setFormatter(logging.Formatter(format_str))
        file_handler.setLevel(log_level)
        logger.addHandler(file_handler)

    return logger

class AvgMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.losses = []

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        self.losses.append(val)

# Data Preprocessing
- 원본 이미지 사이즈가 매우 큰 점을 감안해 (1024,1024)로 resize하여 데이터를 새롭게 저장하였습니다.

In [4]:
df = pd.read_csv('../data/train.csv')

# Resize Train Images
save_path = '../data/train_1024'
os.makedirs(save_path, exist_ok=True)
for img in tqdm(df['img_path']):
    name = os.path.basename(img)
    img = cv2.imread(opj('../data/', img))
    img = cv2.resize(img, dsize=(1024, 1024))
    img = cv2.imwrite(opj(save_path, name), img)

# Resize Test Images
df = pd.read_csv('../data/test.csv')
save_path = '../data/test_1024'
os.makedirs(save_path, exist_ok=True)
for img in tqdm(df['img_path']):
    name = os.path.basename(img)
    img = cv2.imread(opj('../data/', img))
    img = cv2.resize(img, dsize=(1024, 1024))
    img = cv2.imwrite(opj(save_path, name), img)

100%|██████████| 4750/4750 [04:32<00:00, 17.46it/s]


# Dataset & Loader

In [5]:
class Train_Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.img_path = df['img_path'].values
        self.target = df['disease_code'].values 
        self.transform = transform

        print(f'Dataset size:{len(self.img_path)}')

    def __getitem__(self, idx):
        image = cv2.imread(opj('../data/', self.img_path[idx])).astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0
        target = self.target[idx]

        if self.transform is not None:
            image = self.transform(torch.from_numpy(image.transpose(2,0,1)))

        return image, target

    def __len__(self):
        return len(self.img_path)

class Test_dataset(Dataset):
    def __init__(self, df, transform=None):
        self.test_img_path = df['img_path'].values
        self.transform = transform

        print(f'Test Dataset size:{len(self.test_img_path)}')

    def __getitem__(self, idx):
        image = cv2.imread(opj('../data/', self.test_img_path[idx])).astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0

        if self.transform is not None:
            image = self.transform(torch.from_numpy(image.transpose(2,0,1)))

        return image

    def __len__(self):
        return len(self.test_img_path)

def get_loader(df, phase: str, batch_size, shuffle,
               num_workers, transform):
    if phase == 'test':
        dataset = Test_dataset(df, transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True)
    else:
        dataset = Train_Dataset(df, transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True,
                                 drop_last=False)
    return data_loader

def get_train_augmentation(img_size, ver):
    if ver==1: # for validset
        transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                ])

    if ver == 2:
        transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomAffine((20)),
                transforms.RandomRotation(90),
                transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])
    
    
    return transform


# Network

In [6]:
class Network(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.encoder = timm.create_model(args.encoder_name, pretrained=True,
                                    drop_path_rate=args.drop_path_rate,
                                    )
        
        num_head = self.encoder.head.fc.in_features
        self.encoder.head.fc = nn.Linear(num_head, 7)

    def forward(self, x):
        x = self.encoder(x)
        return x

class Network_test(nn.Module):
    def __init__(self, encoder_name):
        super().__init__()
        self.encoder = timm.create_model(encoder_name, pretrained=True,
                                    drop_path_rate=0,
                                    )
        
        num_head = self.encoder.head.fc.in_features
        self.encoder.head.fc = nn.Linear(num_head, 7)
    
    def forward(self, x):
        x = self.encoder(x)
        return x

# Trainer for Training & Validation

In [7]:
class Trainer():
    def __init__(self, args, save_path):
        '''
        args: arguments
        save_path: Model 가중치 저장 경로
        '''
        super(Trainer, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Logging
        log_file = os.path.join(save_path, 'log.log')
        self.logger = get_root_logger(logger_name='IR', log_level=logging.INFO, log_file=log_file)
        self.logger.info(args)
        # self.logger.info(args.tag)

        # Train, Valid Set load
        ############################################################################
        if args.step == 0 :
            df_train = pd.read_csv(opj(args.data_path, 'train.csv'))
        else :
            df_train = pd.read_csv(opj(args.data_path, f'train_{args.step}step.csv'))

        if args.image_type is not None:
            df_train['img_path'] = df_train['img_path'].apply(lambda x:x.replace('train_imgs', args.image_type))
            df_train['img_path'] = df_train['img_path'].apply(lambda x:x.replace('test_imgs', 'test_1024'))

        kf = StratifiedKFold(n_splits=args.Kfold, shuffle=True, random_state=args.seed)
        for fold, (train_idx, val_idx) in enumerate(kf.split(range(len(df_train)), y=df_train['disease_code'])):
            df_train.loc[val_idx, 'fold'] = fold
        val_idx = list(df_train[df_train['fold'] == int(args.fold)].index)

        df_val = df_train[df_train['fold'] == args.fold].reset_index(drop=True)
        df_train = df_train[df_train['fold'] != args.fold].reset_index(drop=True)

        # Augmentation
        self.train_transform = get_train_augmentation(img_size=args.img_size, ver=args.aug_ver)
        self.test_transform = get_train_augmentation(img_size=args.img_size, ver=1)

        # TrainLoader
        self.train_loader = get_loader(df_train, phase='train', batch_size=args.batch_size, shuffle=True,
                                       num_workers=args.num_workers, transform=self.train_transform)
        self.val_loader = get_loader(df_val, phase='train', batch_size=args.batch_size, shuffle=False,
                                       num_workers=args.num_workers, transform=self.test_transform)

        # Network
        self.model = Network(args).to(self.device)
        macs, params = get_model_complexity_info(self.model, (3, args.img_size, args.img_size), as_strings=True,
                                                 print_per_layer_stat=False, verbose=False)
        self.logger.info('{:<30}  {:<8}'.format('Computational complexity: ', macs))
        self.logger.info('{:<30}  {:<8}'.format('Number of parameters: ', params))

        # Loss
        self.criterion = nn.CrossEntropyLoss()
        
        # Optimizer & Scheduler
        self.optimizer = optim.Lamb(self.model.parameters(), lr=args.initial_lr, weight_decay=args.weight_decay)
        
        iter_per_epoch = len(self.train_loader)
        self.warmup_scheduler = WarmUpLR(self.optimizer, iter_per_epoch * args.warm_epoch)

        if args.scheduler == 'step':
            self.scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=args.milestone, gamma=args.lr_factor, verbose=True)
        elif args.scheduler == 'cos':
            tmax = args.tmax # half-cycle 
            self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max = tmax, eta_min=args.min_lr, verbose=True)
        elif args.scheduler == 'cycle':
            self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=args.max_lr, steps_per_epoch=iter_per_epoch, epochs=args.epochs)

        if args.multi_gpu:
            self.model = nn.DataParallel(self.model).to(self.device)

        # Train / Validate
        best_loss = np.inf
        best_acc = 0
        best_epoch = 0
        early_stopping = 0
        start = time.time()
        for epoch in range(1, args.epochs+1):
            self.epoch = epoch

            if args.scheduler == 'cos':
                if epoch > args.warm_epoch:
                    self.scheduler.step()

            # Training
            train_loss, train_acc, train_f1 = self.training(args)

            # Model weight in Multi_GPU or Single GPU
            state_dict= self.model.module.state_dict() if args.multi_gpu else self.model.state_dict()

            # Validation
            val_loss, val_acc, val_f1 = self.validate(args, phase='val')

            # Save models
            if val_loss < best_loss:
                early_stopping = 0
                best_epoch = epoch
                best_loss = val_loss
                best_acc = val_acc
                best_f1 = val_f1

                torch.save({'epoch':epoch,
                            'state_dict':state_dict,
                            'optimizer': self.optimizer.state_dict(),
                            'scheduler': self.scheduler.state_dict(),
                    }, os.path.join(save_path, 'best_model.pth'))
                self.logger.info(f'-----------------SAVE:{best_epoch}epoch----------------')
            else:
                early_stopping += 1

            # Early Stopping
            if early_stopping == args.patience:
                break

        self.logger.info(f'\nBest Val Epoch:{best_epoch} | Val Loss:{best_loss:.4f} | Val Acc:{best_acc:.4f} | Val F1:{best_f1:.4f}')
        end = time.time()
        self.logger.info(f'Total Process time:{(end - start) / 60:.3f}Minute')

    # Training
    def training(self, args):
        self.model.train()
        train_loss = AvgMeter()
        train_acc = 0
        preds_list = []
        targets_list = []

        scaler = grad_scaler.GradScaler()
        for i, (images, targets) in enumerate(tqdm(self.train_loader)):
            images = torch.tensor(images, device=self.device, dtype=torch.float32)
            targets = torch.tensor(targets, device=self.device, dtype=torch.long)
            
            if self.epoch <= args.warm_epoch:
                self.warmup_scheduler.step()

            self.model.zero_grad(set_to_none=True)
            if args.amp:
                with autocast():
                    preds = self.model(images)
                    loss = self.criterion(preds, targets)
                scaler.scale(loss).backward()

                # Gradient Clipping
                if args.clipping is not None:
                    scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)

                scaler.step(self.optimizer)
                scaler.update()

            else:
                preds = self.model(images)
                loss = self.criterion(preds, targets)
                loss.backward()
                nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)
                self.optimizer.step()

            if args.scheduler == 'cycle':
                if self.epoch > args.warm_epoch:
                    self.scheduler.step()

            # Metric
            train_acc += (preds.argmax(dim=1) == targets).sum().item()
            preds_list.extend(preds.argmax(dim=1).cpu().detach().numpy())
            targets_list.extend(targets.cpu().detach().numpy())
            # log
            train_loss.update(loss.item(), n=images.size(0))

        train_acc /= len(self.train_loader.dataset)
        train_f1 = f1_score(np.array(targets_list), np.array(preds_list), average='macro')

        self.logger.info(f'Epoch:[{self.epoch:03d}/{args.epochs:03d}]')
        self.logger.info(f'Train Loss:{train_loss.avg:.3f} | Acc:{train_acc:.4f} | F1:{train_f1:.4f}')
        return train_loss.avg, train_acc, train_f1
            
    # Validation or Dev
    def validate(self, args, phase='val'):
        self.model.eval()
        with torch.no_grad():
            val_loss = AvgMeter()
            val_acc = 0
            preds_list = []
            targets_list = []

            for i, (images, targets) in enumerate(self.val_loader):
                images = torch.tensor(images, device=self.device, dtype=torch.float32)
                targets = torch.tensor(targets, device=self.device, dtype=torch.long)

                preds = self.model(images)
                loss = self.criterion(preds, targets)

                # Metric
                val_acc += (preds.argmax(dim=1) == targets).sum().item()
                preds_list.extend(preds.argmax(dim=1).cpu().detach().numpy())
                targets_list.extend(targets.cpu().detach().numpy())

                # log
                val_loss.update(loss.item(), n=images.size(0))
            val_acc /= len(self.val_loader.dataset)
            val_f1 = f1_score(np.array(targets_list), np.array(preds_list), average='macro')

            self.logger.info(f'{phase} Loss:{val_loss.avg:.3f} | Acc:{val_acc:.4f} | F1:{val_f1:.4f}')
        return val_loss.avg, val_acc, val_f1

# Main

In [8]:
def main(args):
    print('<---- Training Params ---->')
    
    # Random Seed
    seed = args.seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

    save_path = os.path.join(args.model_path, (args.exp_num).zfill(3))
    
    # Create model directory
    os.makedirs(save_path, exist_ok=True)
    Trainer(args, save_path)

    return save_path

# Inference & Make pseudo label set

In [9]:
def predict(encoder_name, test_loader, device, model_path):
    model = Network_test(encoder_name).to(device)
    model.load_state_dict(torch.load(opj(model_path, 'best_model.pth'))['state_dict'])
    model.eval()
    preds_list = []
    with torch.no_grad():
        for images in tqdm(test_loader):
            images = torch.as_tensor(images, device=device, dtype=torch.float32)
            preds = model(images)
            preds = torch.softmax(preds, dim=1)
            preds_list.extend(preds.cpu().tolist())

    return np.array(preds_list)

def ensemble_5fold(model_path_list, test_loader, device):
    predict_list = []
    for model_path in model_path_list:
        prediction = predict(encoder_name= 'regnety_040', test_loader = test_loader, device = device, model_path = model_path)
        predict_list.append(prediction)
    ensemble = (predict_list[0] + predict_list[1] + predict_list[2] + predict_list[3] + predict_list[4])/len(predict_list)

    return ensemble


def make_pseudo_df(train_df, test_df, ensemble, step, threshold = 0.9, z_sample = 500): 
    train_df_copy = train_df.copy()
    test_df_copy = test_df.copy()

    test_df_copy['disease'] = np.nan
    test_df_copy['disease_code'] = ensemble.argmax(axis=1)
    pseudo_test_df = test_df_copy.iloc[np.where(ensemble > threshold)[0]].reset_index(drop=True)
    z_idx  = pseudo_test_df[pseudo_test_df['disease_code'] == 0].sample(n=z_sample, random_state=42).index.tolist()
    ot_idx = pseudo_test_df[pseudo_test_df['disease_code'].isin([*range(1,8)])].index.tolist()
    pseudo_test_df = pseudo_test_df.iloc[z_idx + ot_idx]

    train_df_copy = train_df_copy.append(pseudo_test_df, ignore_index=True).reset_index(drop=True) # reset_index
    # print(f'Make train_{step}step.csv')
    train_df_copy.to_csv(f'../data/train_{step}step.csv', index=False)

# Train & Inference
- 5fold Training -> Inference & Ensemble -> Make or Update Pseudo label set -> Add Dataset(Trainset + Pseudo label set)
- 다음과 과정을 반복하기 때문에 Training과 Inference를 동시에 진행했습니다.

In [10]:
img_size = 288
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sub = pd.read_csv('../data/sample_submission.csv')
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
df_test['img_path'] = df_test['img_path'].apply(lambda x:x.replace('test_imgs', 'test_1024'))
test_transform = get_train_augmentation(img_size=img_size, ver=1)
test_dataset = Test_dataset(df_test, test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

start = 0 # first time : Only Trainset
steps = 6 # Number of pseudo labeling times 
for step in range(start, steps+1): 
    models_path = []
    args.step = step
    for s_fold in range(5): # 5fold
        args.fold = s_fold
        args.exp_num = str(s_fold)
        save_path = main(args)
        models_path.append(save_path)
    ensemble = ensemble_5fold(models_path, test_loader, device)
    make_pseudo_df(df_train, df_test, ensemble, step+1)

# For submission
sub.iloc[:, 1] = ensemble.argmax(axis=1)
sub.to_csv(f'./submission.csv', index=False)

2021-10-25 14:27:11,057 INFO: {'exp_num': '0', 'data_path': '../data', 'Kfold': 5, 'model_path': 'results/', 'image_type': 'train_1024', 'encoder_name': 'regnety_040', 'drop_path_rate': 0.2, 'img_size': 288, 'batch_size': 16, 'epochs': 15, 'optimizer': 'Lamb', 'initial_lr': 5e-06, 'weight_decay': 0.001, 'aug_ver': 2, 'scheduler': 'cycle', 'warm_epoch': 2, 'max_lr': 0.001, 'min_lr': 5e-06, 'tmax': 145, 'patience': 15, 'clipping': None, 'amp': True, 'multi_gpu': False, 'logging': False, 'num_workers': 4, 'seed': 42, 'step': 5, 'fold': 0}


Test Dataset size:4750
<---- Training Params ---->
Dataset size:2853
Dataset size:714


2021-10-25 14:27:11,287 INFO: Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-regnet/regnety_040-f0d569f9.pth)
2021-10-25 14:27:13,799 INFO: Computational complexity:       6.58 GMac
2021-10-25 14:27:13,800 INFO: Number of parameters:           19.57 M 
100%|██████████| 179/179 [01:33<00:00,  1.92it/s]
2021-10-25 14:28:46,849 INFO: Epoch:[001/015]
2021-10-25 14:28:46,849 INFO: Train Loss:2.048 | Acc:0.1272 | F1:0.0922
2021-10-25 14:28:50,790 INFO: val Loss:2.053 | Acc:0.1176 | F1:0.0722
2021-10-25 14:28:51,304 INFO: -----------------SAVE:1epoch----------------
100%|██████████| 179/179 [01:31<00:00,  1.96it/s]
2021-10-25 14:30:22,709 INFO: Epoch:[002/015]
2021-10-25 14:30:22,709 INFO: Train Loss:2.016 | Acc:0.1385 | F1:0.1002
2021-10-25 14:30:26,330 INFO: val Loss:2.009 | Acc:0.1429 | F1:0.0872
2021-10-25 14:30:26,803 INFO: -----------------SAVE:2epoch----------------
100%|██████████| 179/179 [01:31<00:00,  1.95it/s]
2021-10-

<---- Training Params ---->
Dataset size:2853
Dataset size:714


2021-10-25 14:51:11,627 INFO: Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-regnet/regnety_040-f0d569f9.pth)
2021-10-25 14:51:11,752 INFO: Computational complexity:       6.58 GMac
2021-10-25 14:51:11,752 INFO: Number of parameters:           19.57 M 
100%|██████████| 179/179 [01:32<00:00,  1.94it/s]
2021-10-25 14:52:43,803 INFO: Epoch:[001/015]
2021-10-25 14:52:43,804 INFO: Train Loss:2.047 | Acc:0.1206 | F1:0.0903
2021-10-25 14:52:47,285 INFO: val Loss:2.080 | Acc:0.1008 | F1:0.0595
2021-10-25 14:52:47,745 INFO: -----------------SAVE:1epoch----------------
100%|██████████| 179/179 [01:31<00:00,  1.95it/s]
2021-10-25 14:54:19,583 INFO: Epoch:[002/015]
2021-10-25 14:54:19,583 INFO: Train Loss:2.016 | Acc:0.1462 | F1:0.1120
2021-10-25 14:54:23,039 INFO: val Loss:2.024 | Acc:0.1092 | F1:0.0652
2021-10-25 14:54:23,508 INFO: -----------------SAVE:2epoch----------------
100%|██████████| 179/179 [01:31<00:00,  1.96it/s]
2021-10-

...

<---- Training Params ---->
Dataset size:2854
Dataset size:713


2021-10-25 15:38:49,939 INFO: Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-regnet/regnety_040-f0d569f9.pth)
2021-10-25 15:38:50,048 INFO: Computational complexity:       6.58 GMac
2021-10-25 15:38:50,049 INFO: Number of parameters:           19.57 M 
100%|██████████| 179/179 [01:31<00:00,  1.97it/s]
2021-10-25 15:40:21,069 INFO: Epoch:[001/015]
2021-10-25 15:40:21,070 INFO: Train Loss:2.041 | Acc:0.1345 | F1:0.1004
2021-10-25 15:40:24,640 INFO: val Loss:2.065 | Acc:0.1038 | F1:0.0608
2021-10-25 15:40:25,191 INFO: -----------------SAVE:1epoch----------------
100%|██████████| 179/179 [01:30<00:00,  1.97it/s]
2021-10-25 15:41:55,930 INFO: Epoch:[002/015]
2021-10-25 15:41:55,930 INFO: Train Loss:2.021 | Acc:0.1517 | F1:0.1099
2021-10-25 15:41:59,582 INFO: val Loss:2.022 | Acc:0.1206 | F1:0.0751
2021-10-25 15:42:00,109 INFO: -----------------SAVE:2epoch----------------
100%|██████████| 179/179 [01:30<00:00,  1.98it/s]
2021-10-

<---- Training Params ---->
Dataset size:2854
Dataset size:713


2021-10-25 16:02:36,678 INFO: Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-regnet/regnety_040-f0d569f9.pth)
2021-10-25 16:02:36,794 INFO: Computational complexity:       6.58 GMac
2021-10-25 16:02:36,794 INFO: Number of parameters:           19.57 M 
100%|██████████| 179/179 [01:31<00:00,  1.96it/s]
2021-10-25 16:04:07,952 INFO: Epoch:[001/015]
2021-10-25 16:04:07,953 INFO: Train Loss:2.051 | Acc:0.1191 | F1:0.0861
2021-10-25 16:04:11,624 INFO: val Loss:2.054 | Acc:0.1108 | F1:0.0682
2021-10-25 16:04:12,202 INFO: -----------------SAVE:1epoch----------------
100%|██████████| 179/179 [01:31<00:00,  1.96it/s]
2021-10-25 16:05:43,467 INFO: Epoch:[002/015]
2021-10-25 16:05:43,467 INFO: Train Loss:2.026 | Acc:0.1352 | F1:0.0975
2021-10-25 16:05:47,142 INFO: val Loss:1.995 | Acc:0.1318 | F1:0.0851
2021-10-25 16:05:47,653 INFO: -----------------SAVE:2epoch----------------
100%|██████████| 179/179 [01:31<00:00,  1.96it/s]
2021-10-

Make train_6step.csv
